In [ ]:

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from shapely import wkb
from shapely import wkb, Point, LineString
from pyspark.sql import Row
import pyspark.pandas as ps
import math

ps.set_option('compute.default_index_type', 'distributed')
spark = SparkSession.builder.appName("pyspark-notebook").master("spark://spark-master:7077").config("spark.executor.memory", "512m").getOrCreate()

#dataframe = spark.read.format("jdbc").options(url="jdbc:postgresql://db:5432/datascience?user=postgres&password=mantequilla", dbtable="rutas",driver="org.postgresql.Driver").load()
buses = spark.read.option("header",True).csv("./data/buses.csv")
paraderos2 = spark.read.option("header",True).csv("./data/paraderos2.csv")
paraderosrutas = spark.read.option("header",True).csv("./data/paraderosrutas.csv")


In [ ]:

def length(s) -> int:
    return len(s)
def some(item):
    punto_bus = wkb.loads(item["geom_bus"], hex=True)
    punto_paradero = wkb.loads(item["parder_geom"], hex=True)
    distancia = punto_bus.distance(punto_paradero)
    temp = item
    temp["distancia"] = distancia
    return temp
def getMin(row):
    peq = row.min()
    return row.filter(row.distancia == peq) 
def createData(buses, paraderos2, paraderosrutas):
    
    busesrutas = paraderosrutas.join(buses, [buses.vendor == paraderosrutas.cod_sinser]).select(paraderosrutas.simt, 
        buses.bus, buses.fechahora, buses.lat, buses.lng, buses.velocidad, buses.pasajeros, buses.vendor, buses.id, buses.geom.alias("geom_bus"),
        paraderosrutas.servicio, paraderosrutas.nombre_par, paraderosrutas.geom.alias("parder_geom"))
    
    data = busesrutas.join(paraderos2, paraderos2.stop_id == busesrutas.simt, 'inner').select(busesrutas.simt, busesrutas.bus, busesrutas.fechahora, busesrutas.lat, busesrutas.lng, busesrutas.velocidad, busesrutas.pasajeros, busesrutas.vendor, busesrutas.id, busesrutas.geom_bus, busesrutas.servicio, busesrutas.nombre_par, busesrutas.parder_geom, paraderos2.stop_id.alias('parader'), paraderos2.stop_name, paraderos2.stop_lat.alias('parder_lat'), paraderos2.stop_lon.alias('parder_lon'))
    
    #print(busesrutas.to_pandas_on_spark().spark.coalesce(10).shape, paraderos2.to_pandas_on_spark().spark.coalesce(10).shape, data.to_pandas_on_spark().spark.coalesce(10).shape)
    
    geom = data.select(data.columns).coalesce(10)
    data_pd = geom.to_pandas_on_spark()#.coalesce(3)
    data_pd_chunk = data_pd.spark.coalesce(10)
    
    new_geom = data_pd_chunk.apply(some, axis='columns')
    busesparaderos = new_geom.to_spark()
    
    temp_minimos = busesparaderos.groupBy(['fechahora']).agg(F.min(busesparaderos.distancia).alias("md"))
    temp_minimos = busesparaderos.groupBy(['fechahora']).agg(F.min(busesparaderos.distancia).alias("md"))
    temp_minimos.printSchema()
    minimos = temp_minimos.select(temp_minimos.md,temp_minimos.fechahora.alias("fh"))

    resultados = busesparaderos.join(minimos, [busesparaderos.distancia == minimos.md , busesparaderos.fechahora == minimos.fh], 'inner')
    
    return resultados
    



In [ ]:
todo = createData(buses,paraderos2, paraderosrutas)

In [ ]:
def H_Punta(data_k,geo,geo_lat,geo_lon,dataframe_buses,n,S):
    #HORA PUNTA
    Hora_PUNTA = ['2018-06-13 07:00:00+00','2018-06-13 08:59:59+00','2018-06-13 18:00:00+00','2018-06-13 19:59:59+00']
    df_sh = data_k
    if S == 'S1':
        df_sh = data_k[(data_k['fechahora'] >= Hora_PUNTA[0]) & (data_k['fechahora'] <= Hora_PUNTA[1])]
    elif S == 'S2':
        df_sh = data_k[(data_k['fechahora'] >= Hora_PUNTA[2]) & (data_k['fechahora'] <= Hora_PUNTA[3])]
    data_ord = df_sh.sort_values(by=['fechahora'])
    data_ord['fechahora'] = pd.to_datetime(data_ord['fechahora'], infer_datetime_format=True)
    date_list = data_ord['fechahora'].tolist()
    dif_times = []

    for k in range(len(date_list)):
        if k > 0:
            dif_times.append((date_list[k]-date_list[k-1]).total_seconds())

    mean = np.mean(dif_times)
    dataframe_buses['paradero'].append(n)
    dataframe_buses['tiempo_promedio'].append(mean)
    dataframe_buses['geo_parader'].append(geo)
    dataframe_buses['geo_parader_lat'].append(geo_lat)
    dataframe_buses['geo_parader_lon'].append(geo_lon)
    dataframe_buses['zona_h'].append('Punta')
    dataframe_buses['zona_h_num'].append(1)
    dataframe_buses['seccion'].append(S)
    if mean < 600:
        dataframe_buses['calificacion'].append(2)
    elif mean == 600:
        dataframe_buses['calificacion'].append(1)
    else:
        dataframe_buses['calificacion'].append(0)

    return dataframe_buses

def H_Valle(data_k,geo,geo_lat,geo_lon,dataframe_buses,n,S):
    #HORA VALLE
    Hora_VALLE = ['2018-06-13 09:00:00+00','2018-06-13 17:59:59+00','2018-06-13 20:00:00+00','2018-06-13 20:44:59+00']
    df_sh = data_k
    if S == 'S1':
        df_sh = data_k[(data_k['fechahora'] >= Hora_VALLE[0]) & (data_k['fechahora'] <= Hora_VALLE[1])]
    elif S == 'S2':
        df_sh = data_k[(data_k['fechahora'] >= Hora_VALLE[2]) & (data_k['fechahora'] <= Hora_VALLE[3])]
    data_ord = df_sh.sort_values(by=['fechahora'])
    data_ord['fechahora'] = pd.to_datetime(data_ord['fechahora'], infer_datetime_format=True)
    date_list = data_ord['fechahora'].tolist()
    dif_times = []

    for k in range(len(date_list)):
        if k > 0:
            dif_times.append((date_list[k]-date_list[k-1]).total_seconds())

    mean = np.mean(dif_times)
    dataframe_buses['paradero'].append(n)
    dataframe_buses['tiempo_promedio'].append(mean)
    dataframe_buses['geo_parader'].append(geo)
    dataframe_buses['geo_parader_lat'].append(geo_lat)
    dataframe_buses['geo_parader_lon'].append(geo_lon)
    dataframe_buses['zona_h'].append('Valle')
    dataframe_buses['zona_h_num'].append(2)
    dataframe_buses['seccion'].append(S)
    if mean < 600:
        dataframe_buses['calificacion'].append(2)
    elif mean == 600:
        dataframe_buses['calificacion'].append(1)
    else:
        dataframe_buses['calificacion'].append(0)

    return dataframe_buses
def H_Bajo(data_k,geo,geo_lat,geo_lon,dataframe_buses,n,S):
    #HORA BAJO
    Hora_BAJO = ['2018-06-13 00:00:00+00','2018-06-13 06:59:00+00','2018-06-13 20:45:00+00','2018-06-13 23:59:59+00']
    df_sh = data_k
    if S == 'S1':
        df_sh = data_k[(data_k['fechahora'] >= Hora_BAJO[0]) & (data_k['fechahora'] <= Hora_BAJO[1])]
    elif S == 'S2':
        df_sh = data_k[(data_k['fechahora'] >= Hora_BAJO[2]) & (data_k['fechahora'] <= Hora_BAJO[3])]
    data_ord = df_sh.sort_values(by=['fechahora'])
    data_ord['fechahora'] = pd.to_datetime(data_ord['fechahora'], infer_datetime_format=True)
    date_list = data_ord['fechahora'].tolist()
    dif_times = []

    for k in range(len(date_list)):
        if k > 0:
            dif_times.append((date_list[k]-date_list[k-1]).total_seconds())

    mean = np.mean(dif_times)
    dataframe_buses['paradero'].append(n)
    dataframe_buses['tiempo_promedio'].append(mean)
    dataframe_buses['geo_parader'].append(geo)
    dataframe_buses['geo_parader_lat'].append(geo_lat)
    dataframe_buses['geo_parader_lon'].append(geo_lon)
    dataframe_buses['zona_h'].append('Bajo')
    dataframe_buses['zona_h_num'].append(3)
    dataframe_buses['seccion'].append(S)
    if mean < 600:
        dataframe_buses['calificacion'].append(2)
    elif mean == 600:
        dataframe_buses['calificacion'].append(1)
    else:
        dataframe_buses['calificacion'].append(0)

    return dataframe_buses

def H_All(data_k,geo,geo_lat,geo_lon,dataframe_buses,n):
    data_ord = data_k.sort_values(by=['fechahora'])
    data_ord['fechahora'] = pd.to_datetime(data_ord['fechahora'], infer_datetime_format=True)
    date_list = data_ord['fechahora'].tolist()
    dif_times = []

    for k in range(len(date_list)):
        if k > 0:
            dif_times.append((date_list[k]-date_list[k-1]).total_seconds())

    mean = np.mean(dif_times)
    dataframe_buses['paradero'].append(n)
    dataframe_buses['tiempo_promedio'].append(mean)
    dataframe_buses['geo_parader'].append(geo)
    dataframe_buses['geo_parader_lat'].append(geo_lat)
    dataframe_buses['geo_parader_lon'].append(geo_lon)
    if mean < 600:
        dataframe_buses['calificacion'].append(2)
    elif mean == 600:
        dataframe_buses['calificacion'].append(1)
    else:
        dataframe_buses['calificacion'].append(0)

    return dataframe_buses

#*************************
def getPromRegcBus():#Regularidad
    Hour_list_from = ['2018-06-13 00:00:00+00','2018-06-13 01:00:00+00','2018-06-13 02:00:00+00','2018-06-13 03:00:00+00','2018-06-13 04:00:00+00','2018-06-13 05:00:00+00','2018-06-13 06:00:00+00','2018-06-13 07:00:00+00','2018-06-13 08:00:00+00','2018-06-13 09:00:00+00','2018-06-13 10:00:00+00','2018-06-13 11:00:00+00','2018-06-13 12:00:00+00','2018-06-13 13:00:00+00','2018-06-13 14:00:00+00','2018-06-13 15:00:00+00','2018-06-13 16:00:00+00','2018-06-13 18:00:00+00','2018-06-13 19:00:00+00','2018-06-13 20:00:00+00','2018-06-13 21:00:00+00','2018-06-13 22:00:00+00','2018-06-13 23:00:00+00']
    Hour_list_to = ['2018-06-13 00:59:59+00','2018-06-13 01:59:59+00','2018-06-13 02:59:59+00','2018-06-13 03:59:59+00','2018-06-13 04:59:59+00','2018-06-13 05:59:59+00','2018-06-13 06:59:59+00','2018-06-13 07:59:59+00','2018-06-13 08:59:59+00','2018-06-13 09:59:59+00','2018-06-13 10:59:59+00','2018-06-13 11:59:59+00','2018-06-13 12:59:59+00','2018-06-13 13:59:59+00','2018-06-13 14:59:59+00','2018-06-13 15:59:59+00','2018-06-13 16:59:59+00','2018-06-13 18:59:59+00','2018-06-13 19:59:59+00','2018-06-13 20:59:59+00','2018-06-13 21:59:59+00','2018-06-13 22:59:59+00','2018-06-13 23:59:59+00']




    dataframe_ruta = pd.read_csv("./E04I.csv")
    #temp2 = dataframe_ruta.groupby(['parader'], group_keys=True, as_index=True).apply(lambda x: x)
    temp4 = dataframe_ruta.groupby(['bus'], group_keys=True, as_index=True).apply(lambda x: x)
    #print(temp2)
    #temp4 = dataframe_ruta.groupby(['parder_geom'], group_keys=True, as_index=True).apply(lambda x: x)
    #list_bus = temp2.index.levels[0]

    #temp3 = temp2.loc['PA146']
    list_bus = temp4.index.levels[0]
    #print(list_bus)
    #temp5 = temp4.loc['BJFS-70']
    #a = temp2.loc['PA146']['fechahora'].tolist()[0]
    
    #b = a.sort_values(by=['fechahora'])
    #print(temp3)
    
    #w = temp3.groupby(['bus'], group_keys=True, as_index=True).apply(lambda x: x)
    #temp6 = temp5.sort_values(by=['fechahora'])
    #del temp6['parder_geom']
    #del temp6['nombre_par']
    
    #print(temp6)
    dataframe_list = list()
    for i in list_bus:
        x = temp4.loc[i]
        a = x.sort_values(by=['fechahora'])
        del a['nombre_par']
        #del a['parder_geom']
        for j in range(23):
            #print(j)
            df = a[(a['fechahora'] >= Hour_list_from[j]) & (a['fechahora'] <= Hour_list_to[j])]
            #if i == 'BJFS-70':
                #print(df)
            df_g = df.loc[df.groupby('parader')['d'].idxmin()]
            #if i == 'BJFS-70':
                #print(df_g)
            dataframe_list.append(df_g)


    dataframe_buses = {'paradero':[],'tiempo_promedio':[],'geo_parader':[],'geo_parader_lat':[],'geo_parader_lon':[],'calificacion':[]}
    # ESTAN ASI PORQUE NO SE Q WEA CON LOS PUNTEROS
    dataframe_buses_p1 = {'paradero':[],'tiempo_promedio':[],'geo_parader':[],'geo_parader_lat':[],'geo_parader_lon':[],'calificacion':[],'zona_h':[],'zona_h_num':[],'seccion':[]}
    dataframe_buses_p2 = {'paradero':[],'tiempo_promedio':[],'geo_parader':[],'geo_parader_lat':[],'geo_parader_lon':[],'calificacion':[],'zona_h':[],'zona_h_num':[],'seccion':[]}
    dataframe_buses_v1 = {'paradero':[],'tiempo_promedio':[],'geo_parader':[],'geo_parader_lat':[],'geo_parader_lon':[],'calificacion':[],'zona_h':[],'zona_h_num':[],'seccion':[]}
    dataframe_buses_v2 = {'paradero':[],'tiempo_promedio':[],'geo_parader':[],'geo_parader_lat':[],'geo_parader_lon':[],'calificacion':[],'zona_h':[],'zona_h_num':[],'seccion':[]}
    dataframe_buses_b1 = {'paradero':[],'tiempo_promedio':[],'geo_parader':[],'geo_parader_lat':[],'geo_parader_lon':[],'calificacion':[],'zona_h':[],'zona_h_num':[],'seccion':[]}
    dataframe_buses_b2 = {'paradero':[],'tiempo_promedio':[],'geo_parader':[],'geo_parader_lat':[],'geo_parader_lon':[],'calificacion':[],'zona_h':[],'zona_h_num':[],'seccion':[]}


    dataf = pd.concat(dataframe_list)
    
    data_gp = dataf.groupby(['parader'], group_keys=True, as_index=True).apply(lambda x: x)
    #print(data_gp)
    list_parader = data_gp.index.levels[0]
    #mean_list = []
    for n in list_parader:
        data_k = data_gp.loc[n]
        geo = data_k['parder_geom'].iat[0]
        geo_lat = data_k['parder_lat'].iat[0]
        geo_lon = data_k['parder_lon'].iat[0]
        
        H_Punta(data_k,geo,geo_lat,geo_lon,dataframe_buses_p1,n,'S1')
        H_Punta(data_k,geo,geo_lat,geo_lon,dataframe_buses_p2,n,'S2')
        H_Valle(data_k,geo,geo_lat,geo_lon,dataframe_buses_v1,n,'S1')
        H_Valle(data_k,geo,geo_lat,geo_lon,dataframe_buses_v2,n,'S2')
        H_Bajo(data_k,geo,geo_lat,geo_lon,dataframe_buses_b1,n,'S1')
        H_Bajo(data_k,geo,geo_lat,geo_lon,dataframe_buses_b2,n,'S2')
        H_All(data_k,geo,geo_lat,geo_lon,dataframe_buses,n)

   
        
    dataframe_total_busparadero_punta1 = pd.DataFrame(dataframe_buses_p1)
    dataframe_total_busparadero_punta2 = pd.DataFrame(dataframe_buses_p2)
    dataframe_total_busparadero_valle1 = pd.DataFrame(dataframe_buses_v1)
    dataframe_total_busparadero_valle2 = pd.DataFrame(dataframe_buses_v2)
    dataframe_total_busparadero_bajo1 = pd.DataFrame(dataframe_buses_b1)
    dataframe_total_busparadero_bajo2 = pd.DataFrame(dataframe_buses_b2)
    dataframe_total_busparadero = pd.DataFrame(dataframe_buses)
    #print(dataframe_total_busparadero_punta1,dataframe_total_busparadero_punta2,dataframe_total_busparadero_valle1,dataframe_total_busparadero_valle2,dataframe_total_busparadero_bajo1,dataframe_total_busparadero_bajo2,dataframe_total_busparadero)

    dataframe_total_busparadero_punta1.to_csv('E04I_PUNTA1.csv')
    dataframe_total_busparadero_punta2.to_csv('E04I_PUNTA2.csv')
    dataframe_total_busparadero_valle1.to_csv('E04I_VALLE1.csv')
    dataframe_total_busparadero_valle2.to_csv('E04I_VALLE2.csv')
    dataframe_total_busparadero_bajo1.to_csv('E04I_BAJO1.csv')
    dataframe_total_busparadero_bajo2.to_csv('E04I_BAJO2.csv')
    dataframe_total_busparadero.to_csv('E04I_ALL.csv')

def get_Bus_Cant_Prom_Por_Paredero(source):#Frecuenica
    Hour_list_from = ['2018-06-13 00:00:00+00','2018-06-13 01:00:00+00','2018-06-13 02:00:00+00','2018-06-13 03:00:00+00','2018-06-13 04:00:00+00','2018-06-13 05:00:00+00','2018-06-13 06:00:00+00','2018-06-13 07:00:00+00','2018-06-13 08:00:00+00','2018-06-13 09:00:00+00','2018-06-13 10:00:00+00','2018-06-13 11:00:00+00','2018-06-13 12:00:00+00','2018-06-13 13:00:00+00','2018-06-13 14:00:00+00','2018-06-13 15:00:00+00','2018-06-13 16:00:00+00','2018-06-13 18:00:00+00','2018-06-13 19:00:00+00','2018-06-13 20:00:00+00','2018-06-13 21:00:00+00','2018-06-13 22:00:00+00','2018-06-13 23:00:00+00']
    Hour_list_to = ['2018-06-13 00:59:59+00','2018-06-13 01:59:59+00','2018-06-13 02:59:59+00','2018-06-13 03:59:59+00','2018-06-13 04:59:59+00','2018-06-13 05:59:59+00','2018-06-13 06:59:59+00','2018-06-13 07:59:59+00','2018-06-13 08:59:59+00','2018-06-13 09:59:59+00','2018-06-13 10:59:59+00','2018-06-13 11:59:59+00','2018-06-13 12:59:59+00','2018-06-13 13:59:59+00','2018-06-13 14:59:59+00','2018-06-13 15:59:59+00','2018-06-13 16:59:59+00','2018-06-13 18:59:59+00','2018-06-13 19:59:59+00','2018-06-13 20:59:59+00','2018-06-13 21:59:59+00','2018-06-13 22:59:59+00','2018-06-13 23:59:59+00']
    dataframe_ruta = source#.toPandas()
    
    #temp2 = dataframe_ruta.groupby(['parader'], group_keys=True, as_index=True).apply(lambda x: x)
    temp2 = dataframe_ruta.groupBy(['parader']).apply(lambda x: x)
    return 
    #temp4 = dataframe_ruta.groupby(['parder_geom'], group_keys=True, as_index=True).apply(lambda x: x)
    list_paredors = temp2.index.levels[0]
    #temp3 = temp2.loc['PA146']
    #a = temp2.loc['PA146']['fechahora'].tolist()[0]

    dataframe_buses = {'paradero':[],'promedio_bus_hora':[],'geo_parader':[],'geo_parader_lat':[],'geo_parader_lon':[],'calificacion':[]}
    #print(len(Hour_list_from))
    #print(len(Hour_list_to))

    for i in list_paredors: 
        x = temp2.loc[i]
        List_total_buses = list()
        geo = x['parder_geom'].iat[0]
        geo_lat = x['parder_lat'].iat[0]
        geo_lon = x['parder_lon'].iat[0]
        #paradero_list = list()
        #print(geo)
        for j in range(23):
            #print(j)
            df = x[(x['fechahora'] >= Hour_list_from[j]) & (x['fechahora'] <= Hour_list_to[j])]
            List_total_buses.append(df['bus'].nunique())
            #Lista_interval.append(Hour_list_from[j]+' to '+Hour_list_to[j])
        mean = (sum(List_total_buses) / 24)
        dataframe_buses['paradero'].append(i)
        dataframe_buses['promedio_bus_hora'].append(mean)
        dataframe_buses['geo_parader'].append(geo)
        dataframe_buses['geo_parader_lat'].append(geo_lat)
        dataframe_buses['geo_parader_lon'].append(geo_lon)
        if mean < 5:
            dataframe_buses['calificacion'].append(0)
        elif mean > 6:
            dataframe_buses['calificacion'].append(1)
        else:
            dataframe_buses['calificacion'].append(2)


        
        #dataframe_buses['rango_hora'] += Lista_interval


    dataframe_total_busparadero = pd.DataFrame(dataframe_buses)

#print(resultados.to_pandas_on_spark().shape)
#get_Bus_Cant_Prom_Por_Paredero(resultados)

In [ ]:
def processing(group):
    name = group.service
    get_Bus_Cant_Prom_Por_Paredero(name)
    getPromRegcBus(name)
    return group
todo_pandas = todo.to_pandas_on_spark().spark.coalesce(10)
todo_pandas.groupby('servicio').apply(processing, as_index=False, axis='index')